In [1]:
from gerrychain import Graph
from geopy.distance import geodesic
import math
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import geopandas as gpd

In [2]:
# Read Alabama tract graph from the json file "AL_county.json"
filepath = "C:\\Users\\19186\\Downloads\\Alabama_Project\\"
filename = 'AL_tract.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filepath + filename )

In [3]:
# For each node, print the node #, county name, population, and lat-long coordinates
for node in G.nodes:
    name = G.nodes[node]["NAME20"]
    population = G.nodes[node]['P0010001'] 
    G.nodes[node]['TOTPOP'] = population
    
    # query lat and long coordinates 
    G.nodes[node]['C_X'] = G.nodes[node]['INTPTLON20'] #longitude of county's center
    G.nodes[node]['C_Y'] = G.nodes[node]['INTPTLAT20'] #Latitude of county's center

In [4]:
#Get Black Minorty and Voting Age Population
codes = ['P0030004'] # Black or African American alone 
codes += ['P0030011','P0030016','P0030017','P0030018','P0030019'] # Black or African American (among 2 races)
codes += ['P0030027','P0030028','P0030029','P0030030','P0030037','P0030038','P0030039','P0030040','P0030041','P0030042'] # among 3
codes += ['P0030048','P0030049','P0030050','P0030051','P0030052','P0030053','P0030058','P0030059','P0030060','P0030061'] # among 4
codes += ['P0030064','P0030065', 'P0030066','P0030067','P0030069'] # among 5
codes += ['P0030071'] # among 6

#Then, to sum the quantities associated with these codes, I do the following:
for i in G.nodes: 
        
# voting age population (VAP)
    G.nodes[i]['VAP'] = G.nodes[i]['P0030001']
 # Black voting age population (BVAP)
    G.nodes[i]['BVAP'] = sum(G.nodes[i][code] for code in codes )

In [5]:
# create distance dictionary
dist = { (i,j) : 0 for i in G.nodes for j in G.nodes }
for i in G.nodes:
    for j in G.nodes:
        loc_i = (G.nodes[i]['C_Y'], G.nodes[i]['C_X'])
        loc_j = (G.nodes[j]['C_Y'], G.nodes[j]['C_X'])
        dist[i,j] = geodesic(loc_i,loc_j).miles

In [7]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

# number of districts
k = 7          

total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 714166 and U = 721342 and k = 7


In [8]:
# create model 
m = gp.Model()

# create x[i,j] variable which equals one when county i 
# is assigned to (the district centered at) county j
x = m.addVars(G.nodes, G.nodes, vtype=GRB.BINARY) 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-18


In [9]:
# objective is to minimize the moment of inertia: sum (d^2 * p * x over all i and j)
m.setObjective(gp.quicksum( dist[i,j] * dist[i,j] * G.nodes[i]["TOTPOP"] * x[i,j] for i in G.nodes for j in G.nodes))

In [10]:
# add constraints saying that each county i is assigned to one district
m.addConstrs( gp.quicksum( x[i,j] for j in G.nodes ) == 1 for i in G.nodes )


# add constraint saying there should be k district centers
m.addConstr( gp.quicksum( x[j,j] for j in G.nodes ) == k)

# add constraints that say: if j roots a district, then its population is between L and U.
m.addConstrs( gp.quicksum(G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) >= L * x[j,j] for j in G.nodes)
m.addConstrs( gp.quicksum(G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes) <= U * x[j,j] for j in G.nodes)

#add 2 black majority districts
#We will have to decalare the centers because we need these districts to be centered at a tract where we know this will be possible

#Tract in Greenvile in the middle of the black belt
x[1109,1109].LB = 1

#Tract in Tuscaloosa also black belt towards the top
x[5,5].LB = 1

m.addConstr(gp.quicksum(G.nodes[i]['BVAP'] * x[i,1109] for i in G.nodes) >= gp.quicksum(.5 * G.nodes[i]['VAP'] * x[i,1109] for i in G.nodes))
m.addConstr(gp.quicksum(G.nodes[i]['BVAP'] * x[i,5] for i in G.nodes) >= gp.quicksum(.5 * G.nodes[i]['VAP'] * x[i,5] for i in G.nodes))


# add coupling constraints saying that if i is assigned to j, then j is a center.
m.addConstrs( x[i,j] <= x[j,j] for i in G.nodes for j in G.nodes)

m.update()

In [11]:
# Add contiguity constraints
DG = nx.DiGraph(G)

# Add variable f[j,u,v] which equals the amount of flow (originally from j) that is sent across arc (u,v)
f = m.addVars( DG.nodes, DG.edges, vtype=GRB.CONTINUOUS)
M = DG.number_of_nodes()-1

# Add constraint saying that node j cannot receive flow of its own type
m.addConstrs( gp.quicksum( f[j,u,j] for u in DG.neighbors(j) ) == 0 for j in DG.nodes )

# Add constraints saying that node i can receive flow of type j only if i is assigned to j
m.addConstrs( gp.quicksum( f[j,u,i] for u in DG.neighbors(i)) <= M * x[i,j] for i in DG.nodes for j in DG.nodes if i != j )

# If i is assigned to j, then i should consume one unit of j flow. 
#    Otherwise, i should consume no units of j flow.
m.addConstrs( gp.quicksum( f[j,u,i] - f[j,i,u] for u in DG.neighbors(i)) == x[i,j] for i in DG.nodes for j in DG.nodes if i != j )

m.update()

In [ ]:
# solve, making sure to set a 0.00% MIP gap tolerance(!)
m.Params.MIPGap = 0.0
m.optimize()

In [ ]:
# print the objective value
print(m.objVal)

# retrieve the districts and their populations but first get the district "centers"

centers = [j for j in G.nodes if x[j,j].x > 0.5]

districts = [ [i for i in G.nodes if x[i,j].x > 0.5] for j in centers]
district_counties = [ [ G.nodes[i]["NAME20"] for i in districts[j] ] for j in range(k)]
district_populations = [ sum(G.nodes[i]["TOTPOP"] for i in districts[j]) for j in range(k) ]
black_pop = [ sum(G.nodes[i]["BVAP"] for i in districts[j]) for j in range(k) ]
white_pop = [ sum(G.nodes[i]["VAP"] for i in districts[j]) for j in range(k) ]



# print district info
for j in range(k):
    print("District",j,"has population",district_populations[j],"and contains counties",district_counties[j])
    print("it has a minority voting ratio", (black_pop[j] / white_pop[j]) * 100 , '%')
    print("")

In [13]:
# Let's draw it on a map
# Read Alabama tract shapefile from "AL_county.shp"
filepath = "C:\\Users\\19186\\Downloads\\Alabama_Project\\"
filename = 'AL_tract.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filepath + filename )

In [ ]:
# Which district is each county assigned to?
assignment = [ -1 for i in G.nodes ]

labeling = { i : -1 for i in G.nodes }
for j in range(k):
    district = districts[j]
    for i in district:
        labeling[i] = j

# Now add the assignments to a column of the dataframe and map it
node_with_this_geoid = {G.nodes[i]['GEOID20'] : i for i in G.nodes }

# pick a position u in the dataframe
for u in range(G.number_of_nodes()):
    
    geoid = df['GEOID20'][u]
    
    # what node in G has this geoid?
    i = node_with_this_geoid[geoid]
    
    # postion u in the dataframe should be given
    # the same district # that county i has in 'labeling'
    assignment[u] = labeling[i]

#now add the assignents to a column of our dataframe then map it
df['assignment'] = assignment

my_fig = df.plot(column='assignment').get_figure()